In [1]:
root_path = "/tf/home/sergio/Tesis"

In [2]:
import sys
sys.path.append(root_path+"/TinyYOLOv3-Pedestrian-Detection")

from YOLOfunctional import TinyYOLOv3_functional,nms_layer
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.tools import freeze_graph
#from skimage.io import imread,imshow
#from skimage.transform import resize 
import time
#from tensorflow.compat.v1.image import decode_image
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Dataset preparation

In [4]:
image_feature_description = {
    'bboxes': tf.io.FixedLenFeature([], tf.string),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'num_real_boxes':tf.io.FixedLenFeature([], tf.int64),
}

In [5]:
def get_iou_matrix_tf(box_arr1, box_arr2):
    
    box_arr1 = box_arr1 -tf.tile(box_arr1[:,:2],[1,2])
    #print(box_arr1)
    x11, y11, x12, y12 = tf.split(box_arr1, 4, axis=1)
    x21, y21, x22, y22 = tf.split(box_arr2, 4, axis=1)
    xA = tf.maximum(x11, tf.transpose(x21))
    yA = tf.maximum(y11, tf.transpose(y21))
    xB = tf.minimum(x12, tf.transpose(x22))
    yB = tf.minimum(y12, tf.transpose(y22))
    interArea = tf.maximum((xB - xA + 1e-9), 0) * tf.maximum((yB - yA + 1e-9), 0)
    boxAArea = (x12 - x11 + 1e-9) * (y12 - y11 + 1e-9)
    boxBArea = (x22 - x21 + 1e-9) * (y22 - y21 + 1e-9)
    iou = interArea / (boxAArea + tf.transpose(boxBArea) - interArea)
    return iou,tf.argmax(iou,axis=1)#[:,tf.newaxis]


def fill_yolo_output(boxes,grid_size,num_anchors,which_anchor_box,which_anchor_box_index):
    #print(boxes.shape)
    #noobj_mask = tf.ones((1,grid_size*grid_size*num_anchors))
    #print(noobj_mask.shape)
    
    x_min,y_min,x_max,y_max =tf.split(boxes,4,axis=1)

    #Transforma las coordenadas de (xmin,ymin,xmax,ymax) --> (xcenter,ycenter,width,height)
    width = x_max-x_min
    height = y_max-y_min
    x_global =x_min + tf.math.divide(x_max - x_min,2)
    y_global =y_min + tf.math.divide(y_max - y_min,2)
    
    
    x_min_anchor,y_min_anchor,x_max_anchor,y_max_anchor =tf.split(which_anchor_box,4,axis=1)
    
    width_anchor = x_max_anchor-x_min_anchor
    height_anchor = y_max_anchor-y_min_anchor
    x_global_anchor =x_min_anchor + tf.math.divide(x_max_anchor - x_min_anchor,2)
    y_global_anchor =y_min_anchor + tf.math.divide(y_max_anchor - y_min_anchor,2)   

    
    #print("el x original",x_global)
    #print("el y original",y_global)
    #print("el w original",width)
    #print("el h original",height)
    
    #porción de la imagen que hay en cada celda
    pixel_per_grid = tf.math.divide(1.,grid_size)
    #print(pixel_per_grid)
    
    #Obtenemos la coordenada de la celda donde están los boundingboxes
    offset_grid_x = x_global//pixel_per_grid
    offset_grid_y = y_global//pixel_per_grid
    
    #Obtenemos el el centro locacon referencia  al celda encontrada previamente
    x_local =tf.math.floormod(x_global,pixel_per_grid)
    y_local =tf.math.floormod(y_global,pixel_per_grid)
    #print(x_local,y_local)
    
    #Valores tx e ty del groudtruth
    tx = tf.math.log(x_local + 1e-07/(1-x_local))
    ty = tf.math.log(y_local+1e-07/(1-y_local))
    tw = tf.math.log(tf.math.divide(width+1e-07,width_anchor))
    th = tf.math.log(tf.math.divide(height+1e-07,height_anchor))
    tobj_mask = tf.ones_like(tx)
    tobj = tf.concat([tobj_mask,tobj_mask],axis=0)
    
    #tnoobj = tf.zeros_like(tx)    
    #tobj = tf.ones((grid_size*grid_size*num_anchors,1))
    #tnoobj = tf.zeros((grid_size*grid_size*num_anchors,1))
    #print("Lo que la red debe predecir",tx.numpy(),ty.numpy(),tw.numpy(),th.numpy())
    #x_global = (offset_grid_x * pixel_per_grid) + tf.math.sigmoid(tx)
    #y_global = (offset_grid_y * pixel_per_grid) + tf.math.sigmoid(ty)
    #w = width_anchor*tf.math.exp(tw)
    #h = height_anchor*tf.math.exp(th)
    #print("obtnemos el x_real",x_global)
    #print("obtenemos el y_real",y_global)
    #print("obtenemos el w real",w)
    #print("obtenemos el h real",h)
    
    #anchor_boxes_per_output = num_anchors//2

    #Residuo indica cual de los 3 anchor boxes de la coordenada es la que llevara el 1
    #Coord representa la coordenada del grid
    
    residuo = tf.math.floormod(which_anchor_box_index,num_anchors)[:,tf.newaxis]
    coord = tf.cast(num_anchors*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    
    coord_objectness = tf.cast(2*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    coord_objectness2 = coord_objectness+1
    coord_objectess_global = tf.concat([coord_objectness,coord_objectness2],axis=0)
    
    output_position = residuo+coord
    print("tipo de aoutput_positivon",output_position)
    
    print(output_position)
    
    dense_shape = grid_size*grid_size*num_anchors
    print(dense_shape)
    tx_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tx[:,0], dense_shape=[dense_shape]))
    ty_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=ty[:,0], dense_shape=[dense_shape]))
    tw_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tw[:,0], dense_shape=[dense_shape]))
    th_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=th[:,0], dense_shape=[dense_shape]))
    obj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tobj_mask[:,0], dense_shape=[dense_shape]))
    objectness_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=coord_objectess_global, values=tobj[:,0], dense_shape=[dense_shape]))
    #noobj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tnoobj[:,0], dense_shape=[dense_shape]))
    #obj_mask =tx_vector=ty_vector=tw_vector=th_vector = tf.zeros((1,grid_size*grid_size*num_anchors))
    
    tx_vector_dense = tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=False, name="Dense_tx")
    ty_vector_dense = tf.sparse.to_dense(ty_vector, default_value=0, validate_indices=False, name="Dense_ty")
    tw_vector_dense = tf.sparse.to_dense(tw_vector, default_value=0, validate_indices=False, name="Dense_tw")
    th_vector_dense = tf.sparse.to_dense(th_vector, default_value=0, validate_indices=False, name="Dense_th")
    obj_mask_dense =  tf.sparse.to_dense(obj_mask, default_value=0, validate_indices=False, name="Dense_obj")
    #noobj_mask_dense = 1-obj_mask_dense
    objectness_vector_dense =  tf.sparse.to_dense(objectness_vector, default_value=0, validate_indices=False)
    
    #noobj_mask_dense= tf.sparse.to_dense(noobj_mask, default_value=1, validate_indices=False, name="Dense_noobj")
    ##print(tx_vector.to_dense)
    #print(tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=True, name=None)
    #tx_vector=tx_vector[[3,2],]
    #tx_vector[output_position[:,0]] = tx
    #print("coordenada de la salida:",output_position)
    
    #return ((tx_vector_dense,ty_vector_dense,obj_mask_dense),(tw_vector_dense,th_vector_dense,obj_mask_dense),(objectness),(objectness))
    
    return tx_vector_dense,ty_vector_dense,tw_vector_dense,th_vector_dense,obj_mask_dense,objectness_vector_dense

def build_targets(image,image_bboxes,num_real_boxes,anchor_boxes):
    
    images_bboxes_original = image_bboxes
    #Obtenemos los boduing boxes que son reales
    image_bboxes = image_bboxes[:num_real_boxes,:]
    #print("Bouding boxes de la imagen",image_bboxes)
    #Obteneos  la matriz de IoU , y el índice del anchor box que dió mejor resultado
    
    #Nprmalizamos con respecto al tamaño de la imagen y obtenemos la Iou con los anchor boxes
    image_bboxes = tf.math.divide(image_bboxes,416)
    iou_matrix,which_anchor_box_index = get_iou_matrix_tf(image_bboxes,anchor_boxes)
    
    print(which_anchor_box_index)

    anchor_boxes_per_output = len(anchor_boxes)//2
    #Indices de los bouding boxes que irian en cada salida, index_best_ yolo nos dice que bouding boxes de la imagen van a la salida YOLO1,
    #porque su mejor IoU fue con los len(anchor_boxes)//2 anchor boxes mas grandes
    index_best_yolo1 = tf.where(which_anchor_box_index>=anchor_boxes_per_output)[:,0]
    index_best_yolo2 = tf.where(which_anchor_box_index<anchor_boxes_per_output)[:,0]
    index_best_anchor_yolo1 = tf.gather(which_anchor_box_index,index_best_yolo1,axis=0)
    index_best_anchor_yolo2 = tf.gather(which_anchor_box_index,index_best_yolo2,axis=0)
    
    print(index_best_yolo1)
    print(index_best_anchor_yolo1)

    print(index_best_yolo2)
    print(index_best_anchor_yolo2)

    
    best_bboxes_yolo1 = tf.gather(image_bboxes,index_best_yolo1,axis =0)
    best_anchors_yolo1 = tf.gather(anchor_boxes,index_best_anchor_yolo1, axis =0) #LOs dos anchor boxes grandes corrsponden a YOLO1
    best_bboxes_yolo2 = tf.gather(image_bboxes,index_best_yolo2,axis =0)
    best_anchors_yolo2 = tf.gather(anchor_boxes,index_best_anchor_yolo2, axis =0) #Los dos anchor boxes pequeños corresponden a YOLO2
    
    
    if best_anchors_yolo1.shape[0] !=0:
        tx_vector_yolo1,ty_vector_yolo1,tw_vector_yolo1,th_vector_yolo1,obj_mask_yolo1,obj_vector_yolo1= fill_yolo_output(best_bboxes_yolo1,13,anchor_boxes_per_output,best_anchors_yolo1,index_best_anchor_yolo1)
    else:
        tx_vector_yolo1=ty_vector_yolo1=tw_vector_yolo1=th_vector_yolo1=obj_mask_yolo1= obj_vector_yolo1=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo1 = tf.ones((1,13*13*num_anchors))
    
    if best_anchors_yolo2.shape[0] != 0:
        tx_vector_yolo2,ty_vector_yolo2,tw_vector_yolo2,th_vector_yolo2,obj_mask_yolo2,obj_vector_yolo2 = fill_yolo_output(best_bboxes_yolo2,26,anchor_boxes_per_output,best_anchors_yolo2,index_best_anchor_yolo2)
    else:
        tx_vector_yolo2=ty_vector_yolo2=tw_vector_yolo2=th_vector_yolo2=obj_mask_yolo2 = obj_vector_yolo2=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo2 = tf.ones((1,26*26*num_anchors))
        
    tx_vector = tf.concat([tx_vector_yolo1,tx_vector_yolo2],axis=0)[:,tf.newaxis]
    ty_vector = tf.concat([ty_vector_yolo1,ty_vector_yolo2],axis=0)[:,tf.newaxis]
    tw_vector = tf.concat([tw_vector_yolo1,tw_vector_yolo2],axis=0)[:,tf.newaxis]
    th_vector = tf.concat([th_vector_yolo1,th_vector_yolo2],axis=0)[:,tf.newaxis]
    obj_mask = tf.concat([obj_mask_yolo1,obj_mask_yolo2],axis=0)[:,tf.newaxis]
    #noobj_mask = tf.concat([noobj_mask_yolo1,noobj_mask_yolo2],axis=0)[:,tf.newaxis]
    obj_vector = tf.concat([obj_vector_yolo1,obj_vector_yolo2],axis=0)[:,tf.newaxis]
    
    #output = tf.concat([tx_vector,ty_vector,tw_vector,th_vector,obj_mask,noobj_mask,obj_vector],axis=1)
    #images_bboxes_original
    #return image,output
    #Vamos a regresar obj mask que es 1 cuando hay objeto en grid y el anchor box especifico
    return tf.cast(image,tf.float32)/255,(tf.concat([tx_vector,ty_vector,obj_mask],axis=1),tf.concat([tw_vector,th_vector,obj_mask],axis=1),(obj_mask),(obj_mask))

def imgaug_data_augmentation(image,bboxes,num_real_boxes):
    im_shape = image.shape
    bbs = BoundingBoxesOnImage.from_xyxy_array(bboxes*416, shape=(416,416))
    
    policy = np.random.randint(5)
    
    #policy = 2
    if policy == 0:
        
        p = np.random.random()
        if p<=0.6:
            aug = iaa.TranslateX(px=(-60, 60),cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
        p = np.random.random()
        if p<=0.8:
            aug = iaa.HistogramEqualization()
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
    elif policy==1:
        
        p=np.random.random()
        if p<=0.2:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
        
        p=np.random.random()
        if p<=0.8:
            square_size = np.random.randint(48)
            aug = iaa.Cutout(nb_iterations=1, size=square_size/416, squared=True)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    elif policy==2:
        p=np.random.random()
        if p<=1:
            aug = iaa.ShearY(shear=(int(-0.06*416), int(0.06*416)), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
        p=np.random.random()
        if p<=0.6:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
            
    elif policy==3:
        p=np.random.random()
        if p<=0.6:    
            aug = iaa.Rotate(rotate=(-30, 30), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs_aug.remove_out_of_image().clip_out_of_image()
        
        p=np.random.random()
        if p<=1:
            aug = iaa.MultiplySaturation((0.54, 1.54))
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    bbs.remove_out_of_image()
    
    return image,np.clip(bbs.to_xyxy_array(np.float32),1,415),num_real_boxes
    
    
def preprocessing(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,num_real_boxes

def preprocessing_validation_set(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    bboxes = tf.clip_by_value(bboxes*416,1,415)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,tf.cast(num_real_boxes,tf.int64)
    
@tf.function(input_signature=[tf.TensorSpec((416,416,3), tf.uint8),tf.TensorSpec((None,4), tf.float32),tf.TensorSpec((), tf.int64)]) 
def tf_numpy_albumentations_real(image,bboxes,num_real_boxes):
    
    boxes_shape = bboxes.shape
    im_shape = image.shape

    image,bboxes,num_real_boxes = tf.numpy_function(imgaug_data_augmentation,[image,bboxes,num_real_boxes],Tout =[tf.uint8,tf.float32,tf.int64])
 
    image.set_shape(im_shape)
    bboxes.set_shape(boxes_shape)
    print("Imagen data type",image.dtype)
    print("Bboxes data type",bboxes.dtype)
    print("num_real_boxes",num_real_boxes.dtype)

    return image,bboxes,num_real_boxes

In [6]:
#USANDO TF.IMAGE MODULE
#anchors =tf.constant(np.array([[0,0,0.015,0.037],[0,0,0.043,0.104],[0,0,0.11,0.278],[0,0,0.351,0.66]]),dtype=tf.float32)
#anchors =tf.constant(np.array([[0,0,0.026,0.062],[0,0,0.067,0.183],[0,0,0.128,0.323],[0,0,0.343,0.650]]),dtype=tf.float32)
anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)



os.chdir(root_path+"/pedestrian_dataset_train_tfr")
filenames = os. listdir()
raw_image_dataset = tf.data.TFRecordDataset(filenames)

os.chdir(root_path+"/pedestrian_dataset_val_tfr_fixed")
filenames = os. listdir()
raw_image_dataset_val =tf.data.TFRecordDataset(filenames)

os.chdir(root_path+"/pedestrian_dataset_train_tfr")
#.shuffle(70000)

#VAMOS A HACER UN ENTRENAMIENTO SIN DATA AUGMENTATION, ADEMAS DE USAR MSE Y EXP MSE Y REZAR PARA QUE MEJORE

train_dataset = raw_image_dataset.map(preprocessing,num_parallel_calls=8)
train_dataset = train_dataset.map(tf_numpy_albumentations_real,num_parallel_calls=8)
train_dataset = train_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
train_dataset = train_dataset.batch(16)

val_dataset = raw_image_dataset_val.map(preprocessing_validation_set,num_parallel_calls=8)
val_dataset = val_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
val_dataset = val_dataset.batch(16)

Imagen data type <dtype: 'uint8'>
Bboxes data type <dtype: 'float32'>
num_real_boxes <dtype: 'int64'>
Tensor("ArgMax:0", shape=(None,), dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", shape=(None,), dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", shape=(None,), dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", shape=(None, 1), dtype=int64)
Tensor("add_18:0", shape=(None, 1), dtype=int64)
507
tipo de aoutput_positivon Tensor("add_30:0", shape=(None, 1), dtype=int64)
Tensor("add_30:0", shape=(None, 1), dtype=int64)
2028
Tensor("ArgMax:0", dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", dtype=int64)
Tensor("add_18:0", dtype=int64)
507
tipo de aoutput_positivon Tensor("add_30:0", dtype=int64)
Tensor("add_30:0

### Loss Function

In [7]:
from tensorflow.keras.losses import Loss,BinaryCrossentropy,MeanSquaredError,MeanSquaredLogarithmicError
'''
def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h


def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):

    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h
    
def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y
    
def loss_wh(y_true,y_pred):

    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tf.math.exp(tw_true),tf.math.exp(tw_pred))[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tf.math.exp(th_true),tf.math.exp(th_pred))[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h
    
'''


def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y




def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h

def loss_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_no_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''
def loss_bce_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_bce_no_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''

'\ndef loss_bce_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)\n    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   \n    \n    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_obj\n\ndef loss_bce_no_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_noobj\n'

## Loading the functional model

In [8]:
#anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]

model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,training=True)
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights(current_directory+'/weights_functional_one_class/original_model_one_class')

## Training with Model.fit()

In [9]:
for l in model.layers:
    print(l.name, l.trainable)#,l.weights[0].shape)
    pass

input_1 True
conv2d True
batch_normalization True
leaky_re_lu True
max_pooling2d True
conv2d_1 True
batch_normalization_1 True
leaky_re_lu_1 True
max_pooling2d_1 True
conv2d_2 True
batch_normalization_2 True
leaky_re_lu_2 True
max_pooling2d_2 True
conv2d_3 True
batch_normalization_3 True
leaky_re_lu_3 True
max_pooling2d_3 True
conv2d_4 True
batch_normalization_4 True
leaky_re_lu_4 True
max_pooling2d_4 True
conv2d_5 True
batch_normalization_5 True
leaky_re_lu_5 True
zero_padding2d True
max_pooling2d_5 True
conv2d_6 True
batch_normalization_6 True
leaky_re_lu_6 True
conv2d_7 True
batch_normalization_7 True
leaky_re_lu_7 True
conv2d_10 True
batch_normalization_9 True
leaky_re_lu_9 True
tf_op_layer_ResizeBilinear True
Concatenate True
conv2d_8 True
conv2d_11 True
batch_normalization_8 True
batch_normalization_10 True
leaky_re_lu_8 True
leaky_re_lu_10 True
conv2d_9 True
conv2d_12 True
tf_op_layer_Shape True
tf_op_layer_Shape_1 True
tf_op_layer_Shape_2 True
tf_op_layer_Shape_3 True
tf_op_lay

## Eager execution training (For debuging)

In [10]:
debugging = False

if debugging:
    model = TinyYOLOv3(1,anchor_boxes=anchors,train=True)
    model.build(batch_input_shape=(None,416,416,3))
    model.summary()
    model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights");
    optimizer = tf.keras.optimizers.Adam()
    #model_loss = YOLOLoss()

    for epochs in range(1,2,1):
        for (images,y_true) in train_dataset:
            with tf.GradientTape() as tape:
                outputs = model(images)
                print("Tamaño de la etiqueta",y_true[0].shape)
                print("Salida",outputs[0].shape)
                loss_x_y = loss_xy(y_true[0],outputs[0])
                loss_w_h = loss_wh(y_true[1],outputs[1])
                loss_obj = loss_objectness(y_true[2],outputs[2])
                loss_noobj = loss_no_objectness(y_true[3],outputs[3])

                total_loss =loss_x_y+loss_w_h+loss_obj+loss_noobj
            #print(epochs,(total_loss.numpy()))
            grads = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

                #logging.info("{}_train_{}, {}, {}".format(
                #    epoch, batch, total_loss.numpy()))
                
else:
    pass

In [11]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,2,1,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

## Batch 16, con pesos 1,1,1,1 with lr=0.0001 y usando mse para el (x,y) y usando mse para (w,h)

In [12]:
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 507s 110ms/step - loss: 49.3641 - tf_op_layer_split_2_loss: 6.8285 - tf_op_layer_split_2_1_loss: 2.5779 - Concatenate_obj_loss: 0.0670 - Concatenate_obj_1_loss: 30.4845 - Concatenate_obj_precision: 0.0440 - Concatenate_obj_recall: 0.0504 - Concatenate_obj_true_positives: 13621.0000 - Concatenate_obj_false_positives: 295982.0000 - val_loss: 29.0434 - val_tf_op_layer_split_2_loss: 5.5292 - val_tf_op_layer_split_2_1_loss: 0.9659 - val_Concatenate_obj_loss: 0.0511 - val_Concatenate_obj_1_loss: 16.0022 - val_Concatenate_obj_precision: 0.7485 - val_Concatenate_obj_recall: 0.0539 - val_Concatenate_obj_true_positives: 637.0000 - val_Concatenate_obj_false_positives: 214.0000
Epoch 2/10
4607/4607 [==============================] - 509s 110ms/step - loss: 28.4992 - tf_op_layer_split_2_loss: 5.4355 - tf_op_layer_split_2_1_loss: 1.9554 - Concatenate_obj_loss: 0.0403 - Concatenate_obj_1_loss: 13.6773 - Concatenate_obj_precision: 0.6403 - Concat

In [13]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [14]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_10_epochs')

## CONTINUACIÓN

In [8]:
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]
model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,training=True)
#print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_10_epochs')

In [9]:
for l in model.layers:
    print(l.name, l.trainable)
    pass

input_1 True
conv2d True
batch_normalization True
leaky_re_lu True
max_pooling2d True
conv2d_1 True
batch_normalization_1 True
leaky_re_lu_1 True
max_pooling2d_1 True
conv2d_2 True
batch_normalization_2 True
leaky_re_lu_2 True
max_pooling2d_2 True
conv2d_3 True
batch_normalization_3 True
leaky_re_lu_3 True
max_pooling2d_3 True
conv2d_4 True
batch_normalization_4 True
leaky_re_lu_4 True
max_pooling2d_4 True
conv2d_5 True
batch_normalization_5 True
leaky_re_lu_5 True
zero_padding2d True
max_pooling2d_5 True
conv2d_6 True
batch_normalization_6 True
leaky_re_lu_6 True
conv2d_7 True
batch_normalization_7 True
leaky_re_lu_7 True
conv2d_10 True
batch_normalization_9 True
leaky_re_lu_9 True
tf_op_layer_ResizeBilinear True
tf_op_layer_Concatenate True
conv2d_8 True
conv2d_11 True
batch_normalization_8 True
batch_normalization_10 True
leaky_re_lu_8 True
leaky_re_lu_10 True
conv2d_9 True
conv2d_12 True
tf_op_layer_Shape True
tf_op_layer_Shape_1 True
tf_op_layer_Shape_2 True
tf_op_layer_Shape_3 Tr

In [10]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,2,1,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [11]:
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 518s 112ms/step - loss: 23.7484 - tf_op_layer_split_2_loss: 4.3206 - tf_op_layer_split_2_1_loss: 1.4996 - Concatenate_obj_loss: 0.0361 - Concatenate_obj_1_loss: 12.0720 - Concatenate_obj_precision: 0.6575 - Concatenate_obj_recall: 0.1623 - Concatenate_obj_true_positives: 43884.0000 - Concatenate_obj_false_positives: 22858.0000 - val_loss: 22.1258 - val_tf_op_layer_split_2_loss: 3.6276 - val_tf_op_layer_split_2_1_loss: 0.6833 - val_Concatenate_obj_loss: 0.0439 - val_Concatenate_obj_1_loss: 13.4602 - val_Concatenate_obj_precision: 0.7943 - val_Concatenate_obj_recall: 0.1228 - val_Concatenate_obj_true_positives: 1452.0000 - val_Concatenate_obj_false_positives: 376.0000
Epoch 2/10
4607/4607 [==============================] - 516s 112ms/step - loss: 23.3009 - tf_op_layer_split_2_loss: 4.2470 - tf_op_layer_split_2_1_loss: 1.4026 - Concatenate_obj_loss: 0.0357 - Concatenate_obj_1_loss: 11.9659 - Concatenate_obj_precision: 0.6588 - Concat

In [12]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [13]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_10_20_epochs')

## eNTRENAMIENTO 10-20 EPOCAS

In [8]:
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]
model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,training=True)
#print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_10_20_epochs')

In [9]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.RMSprop(learning_rate=1e-4)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,2,1,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [10]:
#val_loss: 15.5969 - val_output_1_loss: 0.8771 - val_output_2_loss: 0.2821 - val_output_3_loss: 0.0319 - val_output_4_loss: 9.7367

In [11]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 522s 113ms/step - loss: 21.2655 - tf_op_layer_split_2_loss: 3.7062 - tf_op_layer_split_2_1_loss: 1.3169 - Concatenate_obj_loss: 0.0338 - Concatenate_obj_1_loss: 11.1854 - Concatenate_obj_precision: 0.6812 - Concatenate_obj_recall: 0.2136 - Concatenate_obj_true_positives: 57716.0000 - Concatenate_obj_false_positives: 27012.0000 - val_loss: 18.7153 - val_tf_op_layer_split_2_loss: 2.6984 - val_tf_op_layer_split_2_1_loss: 0.5771 - val_Concatenate_obj_loss: 0.0400 - val_Concatenate_obj_1_loss: 12.1243 - val_Concatenate_obj_precision: 0.8269 - val_Concatenate_obj_recall: 0.1669 - val_Concatenate_obj_true_positives: 1973.0000 - val_Concatenate_obj_false_positives: 413.0000loss: 11.2657 - Concatenate_obj_p
Epoch 2/10
4607/4607 [==============================] - 511s 111ms/step - loss: 21.1879 - tf_op_layer_split_2_loss: 3.6652 - tf_op_layer_split_2_1_loss: 1.3647 - Concatenate_obj_loss: 0.0336 - Concatenate_obj_1_loss: 11.0945 - Concatena

In [12]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [13]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_20_30_epochs')

In [14]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 512s 111ms/step - loss: 19.5762 - tf_op_layer_split_2_loss: 3.3312 - tf_op_layer_split_2_1_loss: 1.2314 - Concatenate_obj_loss: 0.0319 - Concatenate_obj_1_loss: 10.4192 - Concatenate_obj_precision: 0.7109 - Concatenate_obj_recall: 0.2672 - Concatenate_obj_true_positives: 72219.0000 - Concatenate_obj_false_positives: 29363.0000 - val_loss: 15.9157 - val_tf_op_layer_split_2_loss: 2.1207 - val_tf_op_layer_split_2_1_loss: 0.4635 - val_Concatenate_obj_loss: 0.0361 - val_Concatenate_obj_1_loss: 10.7111 - val_Concatenate_obj_precision: 0.8586 - val_Concatenate_obj_recall: 0.2275 - val_Concatenate_obj_true_positives: 2690.0000 - val_Concatenate_obj_false_positives: 443.0000
Epoch 2/10
4607/4607 [==============================] - 513s 111ms/step - loss: 19.5244 - tf_op_layer_split_2_loss: 3.3485 - tf_op_layer_split_2_1_loss: 1.2254 - Concatenate_obj_loss: 0.0317 - Concatenate_obj_1_loss: 10.3449 - Concatenate_obj_precision: 0.7148 - Concat

In [15]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [16]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_30_40_epochs')

In [17]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 520s 113ms/step - loss: 18.7244 - tf_op_layer_split_2_loss: 3.1369 - tf_op_layer_split_2_1_loss: 1.3118 - Concatenate_obj_loss: 0.0303 - Concatenate_obj_1_loss: 9.7968 - Concatenate_obj_precision: 0.7369 - Concatenate_obj_recall: 0.3157 - Concatenate_obj_true_positives: 85306.0000 - Concatenate_obj_false_positives: 30462.0000 - val_loss: 14.3368 - val_tf_op_layer_split_2_loss: 1.8055 - val_tf_op_layer_split_2_1_loss: 0.4417 - val_Concatenate_obj_loss: 0.0338 - val_Concatenate_obj_1_loss: 9.8087 - val_Concatenate_obj_precision: 0.9055 - val_Concatenate_obj_recall: 0.2797 - val_Concatenate_obj_true_positives: 3307.0000 - val_Concatenate_obj_false_positives: 345.0000
Epoch 2/10
4607/4607 [==============================] - 520s 113ms/step - loss: 18.5417 - tf_op_layer_split_2_loss: 3.1512 - tf_op_layer_split_2_1_loss: 1.2337 - Concatenate_obj_loss: 0.0302 - Concatenate_obj_1_loss: 9.7417 - Concatenate_obj_precision: 0.7401 - Concatena

In [18]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [19]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_40_50_epochs')

In [20]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 516s 112ms/step - loss: 17.8119 - tf_op_layer_split_2_loss: 2.9748 - tf_op_layer_split_2_1_loss: 1.2627 - Concatenate_obj_loss: 0.0290 - Concatenate_obj_1_loss: 9.3080 - Concatenate_obj_precision: 0.7572 - Concatenate_obj_recall: 0.3534 - Concatenate_obj_true_positives: 95517.0000 - Concatenate_obj_false_positives: 30635.0000 - val_loss: 13.9047 - val_tf_op_layer_split_2_loss: 1.6999 - val_tf_op_layer_split_2_1_loss: 0.4492 - val_Concatenate_obj_loss: 0.0333 - val_Concatenate_obj_1_loss: 9.5732 - val_Concatenate_obj_precision: 0.9293 - val_Concatenate_obj_recall: 0.2978 - val_Concatenate_obj_true_positives: 3521.0000 - val_Concatenate_obj_false_positives: 268.0000
Epoch 2/10
4607/4607 [==============================] - 509s 110ms/step - loss: 17.7091 - tf_op_layer_split_2_loss: 2.9850 - tf_op_layer_split_2_1_loss: 1.2111 - Concatenate_obj_loss: 0.0290 - Concatenate_obj_1_loss: 9.2879 - Concatenate_obj_precision: 0.7572 - Concatena

In [21]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [22]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_50_60_epochs')

In [23]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
4607/4607 [==============================] - 511s 111ms/step - loss: 16.9920 - tf_op_layer_split_2_loss: 2.8917 - tf_op_layer_split_2_1_loss: 1.1488 - Concatenate_obj_loss: 0.0278 - Concatenate_obj_1_loss: 8.8830 - Concatenate_obj_precision: 0.7712 - Concatenate_obj_recall: 0.3875 - Concatenate_obj_true_positives: 104725.0000 - Concatenate_obj_false_positives: 31065.0000 - val_loss: 12.0608 - val_tf_op_layer_split_2_loss: 1.4654 - val_tf_op_layer_split_2_1_loss: 0.4332 - val_Concatenate_obj_loss: 0.0290 - val_Concatenate_obj_1_loss: 8.2344 - val_Concatenate_obj_precision: 0.9229 - val_Concatenate_obj_recall: 0.3704 - val_Concatenate_obj_true_positives: 4380.0000 - val_Concatenate_obj_false_positives: 366.0000
Epoch 2/10
4607/4607 [==============================] - 513s 111ms/step - loss: 16.9798 - tf_op_layer_split_2_loss: 2.8872 - tf_op_layer_split_2_1_loss: 1.1344 - Concatenate_obj_loss: 0.0279 - Concatenate_obj_1_loss: 8.9087 - Concatenate_obj_precision: 0.7706 - Concaten

In [24]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [25]:
model.save_weights(current_directory+'/weights_fine_tuning_functional/rmsprop_2211_60_70_epochs')

## CONTNUACION DESDE EPOCA 80

In [7]:
anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
#print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
model.load_weights(root_path+'/last_weights/pesos_finetuning_1121_70_80_epoch_nadam_0dot00001_mse_mse_3anchors_con_data_aug')

Modo entrenamiento


In [8]:
for l in model.layers:
    print(l.name, l.trainable)
    pass

BasicBlock1 True
BasicBlock2 True
BasicBlock3 True
BasicBlock4 True
BasicBlock5 True
BasicBlock6 True
BasicBlock7 True
BasicBlock8 True
BasicBlock9 True
FinalBlock1 True
BasicBlock11 True
BasicBlock12 True
FinalBlock2 True
Concatenate True
Upsampling True
Prediction1 True
Prediction2 True
Concatenate_BBOX True


In [9]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=3e-4)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,1,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [10]:
#val_loss: 15.5969 - val_output_1_loss: 0.8771 - val_output_2_loss: 0.2821 - val_output_3_loss: 0.0319 - val_output_4_loss: 9.7367

In [11]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
Modo entrenamiento
Modo entrenamiento
   4607/Unknown - 556s 121ms/step - loss: 10.1637 - output_1_loss: 2.7599 - output_2_loss: 1.0363 - output_3_loss: 0.0196 - output_4_loss: 6.3282 - output_3_precision: 0.8569 - output_3_recall: 0.5845 - output_3_true_positives: 157983.0000 - output_3_false_positives: 26391.0000Modo entrenamiento
4607/4607 [==============================] - 564s 122ms/step - loss: 10.1637 - output_1_loss: 2.7599 - output_2_loss: 1.0363 - output_3_loss: 0.0196 - output_4_loss: 6.3282 - output_3_precision: 0.8569 - output_3_recall: 0.5845 - output_3_true_positives: 157983.0000 - output_3_false_positives: 26391.0000 - val_loss: 6.3997 - val_output_1_loss: 1.4494 - val_output_2_loss: 0.3623 - val_output_3_loss: 0.0164 - val_output_4_loss: 4.5553 - val_output_3_precision: 0.9776 - val_output_3_recall: 0.5793 - val_output_3_true_positives: 6850.0000 - val_output_3_false_positives: 157.0000
Epoch 2/10
4607/4607 [==============================] - 552s 120ms/step 

In [12]:
os.chdir(root_path+"/json_final_experiment")
import json
#json.dumps(str(a))
with open('history_finetuning_1121_80_90_epoch_nadam_0dot00001_mse_mse_3anchors_con_data_aug.json', 'w') as fp:
    json.dump(str(history.history), fp)

In [13]:
model.save_weights(root_path+'/last_weights/pesos_finetuning_1121_80_90_epoch_nadam_0dot00001_mse_mse_3anchors_con_data_aug')

# CONTINUACION EPOCAS DE LA 90 A LA 100

In [7]:
anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)
model = TinyYOLOv3(1,anchor_boxes=anchors,train=True,mode = "finetuning")
model.build(batch_input_shape=(None,416,416,3))
#print(model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights"))
model.load_weights(root_path+'/last_weights/pesos_finetuning_1121_80_90_epoch_nadam_0dot00001_mse_mse_3anchors_con_data_aug')

Modo entrenamiento


In [8]:
tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(learning_rate=3e-4)

losses = {"output_1": loss_xy,
          "output_2": loss_wh,
          "output_3":loss_objectness,
          "output_4":loss_no_objectness  
}

metrics = {"output_3":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[1,1,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [9]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=10,validation_data=val_dataset)

Epoch 1/10
Modo entrenamiento
Modo entrenamiento
   4607/Unknown - 579s 126ms/step - loss: 9.8747 - output_1_loss: 2.6954 - output_2_loss: 1.0173 - output_3_loss: 0.0189 - output_4_loss: 6.1242 - output_3_precision: 0.8630 - output_3_recall: 0.6006 - output_3_true_positives: 162368.0000 - output_3_false_positives: 25780.0000- 546s 126ms/step - loss: 9.8828 - output_1_loss: 2.7033 - output_2_loss: 1.01 - 566s 125ms/step - loss: 9.9384 - output_1_loss: 2.7149 - output_2_loss: 1.0277 - output_3_loss: 0.0191 - output_4_loss: 6.1577 - output_3_precision: 0.8631 - outputModo entrenamiento
4607/4607 [==============================] - 586s 127ms/step - loss: 9.8747 - output_1_loss: 2.6954 - output_2_loss: 1.0173 - output_3_loss: 0.0189 - output_4_loss: 6.1242 - output_3_precision: 0.8630 - output_3_recall: 0.6006 - output_3_true_positives: 162368.0000 - output_3_false_positives: 25780.0000 - val_loss: 5.8917 - val_output_1_loss: 1.4194 - val_output_2_loss: 0.3791 - val_output_3_loss: 0.0144 - 

In [10]:
os.chdir(root_path+"/json_final_experiment")
import json
#json.dumps(str(a))
with open('history_finetuning_1121_90_100_epoch_nadam_0dot00001_mse_mse_3anchors_con_data_aug.json', 'w') as fp:
    json.dump(str(history.history), fp)

In [11]:
model.save_weights(root_path+'/last_weights/pesos_finetuning_1121_90_100_epoch_nadam_0dot00001_mse_mse_3anchors_con_data_aug')